# Майнор по Анализу Данных, Группа ИАД-4
## 21/09/2017 Методы Оптимизации

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

try:
    from ipywidgets import interact, IntSlider, fixed, FloatSlider
except ImportError:
    print(u'Так надо')

# Но в начале...
Еще пару слов про SVM

SVM позволяет встраивать собственные ядра! И это вам пригодится в домашке.

In [ ]:
# Не помню, откуда взял этот код...

from sklearn import svm, datasets

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features. We could
                      # avoid this ugly slicing by using a two-dim dataset
y = iris.target


def my_kernel(U, V):
    """
    We create a custom kernel:

                 (2  0)
    k(U, V) = U  (    ) V.T
                 (0  1)
    """
    M = np.array([[2, 0], [0, 1.0]])
    return np.dot(np.dot(U, M), V.T)


h = .02  # step size in the mesh

# we create an instance of SVM and fit out data.
clf = svm.SVC(kernel=my_kernel)
clf.fit(X, y)

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, m_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.title('3-Class classification using Support Vector Machine with custom'
          ' kernel')
plt.axis('tight')

# Методы оптимизации

Как было показано на лекции, большинство методов машинного обучения сводятся к поиску параметров, которые минимизируют ошибку на тренировочной выборке:
$$
\min_{\beta} L(\beta; D)
$$
Здесь:
* $D$ — размеченная обучающая выборка, $\{x^{(i)}, y^{(i)}\}_{i=1}^N$
* $L$ — функция потерь
* $\beta$ — настраиваемые веса алгоритма

В более общем виде задачу можно записать так:
$$
\min_{x} f(x)
$$
Здесь:
* $x$ — вектор значений
* $f$ — функция, принимающая вектор в качестве аргумента и выдающая числовое значение.

На семинаре рассмотрим подробнее методы минимизации функции, которые рассматривались на лекции.

# Градиентный спуск

Для оптимизации возьмем простую функцию $f(x) = x^3 - 2x^2 + 2$

In [ ]:
f = lambda x: x ** 3 - 2*x ** 2 + 2
df = lambda x: 3 * x ** 2 - 4 * x # производная
x = np.linspace(-1, 2.5, 1000)
plt.plot(x, f(x))
plt.xlim([-1, 2.5])
plt.ylim([0, 3])

И определим функцию, которая будет оптимизировать функцию $f(x)$ градиентным спуском с заданным постоянным шагом (он же learning rate, темп обучения).

In [ ]:
def optimize_and_plot_steps(learning_rate, x_new=2, compute_learning_rate=None):
    x_old = 0
    # x_new — точка старта
    eps = 0.0001
    x_list, y_list = [x_new], [f(x_new)] # инициализируем список координат и значений функций при итерации
    
    # спускаемся, пока разница между координатами не достигла требуемой точности
    i = 0
    while abs(x_new - x_old) > eps: 
        x_old = x_new
        # считаем направление спуска
        direction = -df(x_old)
        # обновляем значение темпа обучения, если нам задана функция для этого
        if compute_learning_rate is not None:
            learning_rate = compute_learning_rate(i, learning_rate)
        # делаем шаг
        x_new = x_old + learning_rate * direction
        # запоминаем очередной шаг минимизации
        x_list.append(x_new)
        y_list.append(f(x_new))
        i += 1
        
    print "Найденный локальный минимум:", x_new
    print "Количество шагов:", len(x_list)
    
    plt.figure(figsize=[10,3])
    
    plt.subplot(1,2,1)
    plt.scatter(x_list, y_list, c="r")
    plt.plot(x_list, y_list, c="r")
    plt.plot(x, f(x), c="b")
    plt.xlim([-1,2.5])
    plt.ylim([0,3])
    plt.title("Descent trajectory")

    plt.subplot(1,2,2)
    plt.scatter(x_list,y_list,c="r")
    plt.plot(x_list,y_list,c="r")
    plt.plot(x,f(x), c="b")
    plt.xlim([1.2,2.1])
    plt.ylim([0,3])
    plt.title("Descent trajectory (zoomed in)")

Попробуем оптимизацию с шагом 0.1

In [ ]:
optimize_and_plot_steps(0.1)

Возьмем шаг побольше.

In [ ]:
optimize_and_plot_steps(0.4)

Что, если взять 0.5?

In [ ]:
optimize_and_plot_steps(0.5)

Застопорились в нуле, т.к. нашли точный локальный максимум. В нем производная равна нулю и мы никуда не можем сдвинуться. А если взять 0.49?

In [ ]:
optimize_and_plot_steps(0.49)

Что, если взять 0.51?

In [ ]:
optimize_and_plot_steps(0.51)

Мы улетели далеко влево. Это можно понять, распечатав значения x_new.

Теперь возьмём маленький шаг. Например, 0.05.

In [ ]:
optimize_and_plot_steps(0.05)

0.01?

In [ ]:
optimize_and_plot_steps(0.01)

Чем меньше шаг, тем медленнее мы идём к минимум (и можем вдобавок застрять по пути). Чем больше темп обучения, тем большие расстояния мы перепрыгиваем (и имеем гипотетическую возможность найти минимум получше). Хорошая стратегия — начинать с достаточно большого шага (чтобы хорошо попутешествовать по функции), а потом постепенно его уменьшать, чтобы стабилизировать процесс обучения в каком-то локальном минимуме.

Теперь будем изменять шаг динамически:
$lr(i + 1) = lr(i) * 0.9$.

In [ ]:
def compute_learning_rate(i, prev_lr):
    return prev_lr * 0.9

In [ ]:
optimize_and_plot_steps(0.4, compute_learning_rate=compute_learning_rate)

Если сравнивать с постоянным темпом обучения, то мы нашли минимум в 2 раза быстрее.

In [ ]:
optimize_and_plot_steps(0.4)

Это, конечно, искуственный пример, но такая же идея используются для обучения алгоритмов машинного обучения с миллионами параметров, функции потерь которых имеют очень сложную структуру и не поддаются визуализации.

In [ ]:
def gradient_descent(X, y, iters, alpha):
    costs = []
    n = y.shape[0] 
    Beta = np.random.rand(2)
    history = [Beta] 
    preds = []
    for i in xrange(iters):
        y_hat = X.dot(Beta)
        error = y_hat - y 
        cost = np.mean(error ** 2) / 2
        
        if i % 100 == 0: 
            preds.append(y_hat)
            history.append(Beta)
            costs.append(cost)

        gradient = X.T.dot(error)
        Beta = Beta - alpha / n * gradient  # update
        
        
    return history, costs, preds, Beta

In [ ]:
res = gradient_descent(X, y, 1000, 0.05)

# Градиентный спуск для линейной регрессии

## Описание задачи линейной регрессии

Напомним суть метода градиентого спуска в контексте задачи линейной регрессии.

Дано описание $n$ объектов по $m$ признакам. Обычно оно выражается в виде матрицы размера $n \times m$: $X = [x^{(i)}_j]^{i=1\dots n}_{j=1\dots m} $.<br\> ($x^{(i)}_j$ означает $j$-ый признак $i$-го объекта) <br\>
Дана зависимая переменная, которая тоже имеет отношение к этим объекам: $y$ - вектор длины $n$.

Наша задача, выявить **линейную** зависимость между признаками в $X$ и значениями в $y$:
$$\hat{y} = X\beta \quad \Leftrightarrow \quad \hat{y}^{(i)} = \beta_0 + \beta_1x^{(i)}_1 + \dots$$

И сделать это так, чтобы квадрат суммы ошибок наших оценок был минимален:
$$ L(\beta) = \frac{1}{2n}(\hat{y} - y)^{\top}(\hat{y} - y) = \frac{1}{2n}(X\beta - y)^{\top}(X\beta - y) \rightarrow \min$$ $$ \Updownarrow $$  $$ L(\beta_0,\beta_1,\dots) = \frac{1}{2n}\sum^{n}_{i=1}(\hat{y}^{(i)} - y^{(i)})^2 = \frac{1}{2n}\sum^{n}_{i=1}(\beta_0 + \beta_1x^{(i)}_1 + \dots - y^{(i)})^2  \rightarrow \min $$

Значение в $X$ и $y$ нам даны. Нам неизвестны только значения коэффициентов $\beta$.<br\> Соответственно, нужно найти такие значения $\beta$, что функция $L(\beta) \rightarrow \min.$

## Описание метода градиентного спуска для регрессии

Пусть нам известнен 1 признак объекта и мы включаем свободный член у ровнение регрессии.

Посчитаем, чему равен градиент функции потерь $L(\beta_0, \beta_1):$
$$ \frac{\partial L}{\partial \beta_0} = \frac{1}{n}\sum^{n}_{i=1}(\beta_0 + \beta_1x^{(i)}_1 - y^{(i)})$$
$$ \frac{\partial L}{\partial \beta_1} = \frac{1}{n}\sum^{n}_{i=1}(\beta_0 + \beta_1x^{(i)}_1 - y^{(i)})x_1^{(i)}$$

Иногда проще это записать в виде матриц:
$$ \nabla_\beta L(\beta) = X^\top(X\beta - y)$$


Метод градиентного спуска заключается в итеративном и **одновременном(!!!)** обновлении значений $\beta$ в направлении, противоположному градиенту:
$$ \beta := \beta - \alpha\nabla_\beta L(\beta)$$

## Пошаговое описание программной реализации

Для начала нам понадобятся данные. 

Используем те же данные про грузовики, что и были даны на втором семинаре. Нам дано два столбца значений — количество жителей в городе и доход грузовика с уличной едой в этом городе.

Будем строить модель, описывающую зависимость дохода от размера населения.

In [ ]:
!head ./data/food_trucks.txt

In [ ]:
filepath = os.path.join('data', 'food_trucks.txt')
data = np.loadtxt(filepath, delimiter=',')
plt.scatter(data[:,0], data[:, 1])
plt.xlabel('population')
plt.ylabel('income')

In [ ]:
# Тогда X - это будет матрица размера ( 97x2 ), a y - вектор-столбец

# Отнормируем данные
data[:, 0] = (data[:, 0] - data[:, 0].mean())/data[:, 0].std()
X = np.c_[data[:, 0], np.ones((data.shape[0], 1))]

y = data[:, 1]

Теперь научимся считать ошибку модели. Для этого нам нужны коэффициенты. 
Пока возьмем некоторые случайные значения

In [ ]:
Beta = np.array([5,6])

Как было показано выше: 
$$ \hat{y}^{(i)} = \beta_0 + \beta_1x^{(i)}_1 + \dots \quad \Leftrightarrow \quad \hat{y} = X\beta \quad \Leftrightarrow \quad \texttt{y_hat = X.dot(Beta)} $$

In [ ]:
y_hat = X.dot(Beta)

Соответственно, ошибка это $(X\beta - y)$, а функция потерь $ L(\beta) = \frac{1}{2n}(X\beta - y)^{\top}(X\beta - y) = \frac{1}{2n}\sum^{n}_{i=1}(\beta_0 + \beta_1x^{(i)}_1 + \dots - y^{(i)})^2 $

In [ ]:
error = (X.dot(Beta) - y)

n = X.shape[0]
cost = np.sum(error ** 2) / (2 * n)

Отлично, мы научились считать ошибку при заданных $\beta$. Теперь выразим градиент:

$$ \nabla_\beta L(\beta)= X^\top(X\beta - y) \quad \Leftrightarrow \quad  \texttt{grad = X.T.dot(error)} $$

Теперь к шагам алгоритма:

* Задаем случайное начальное значение для $\beta$
* Пока не будет достигнуто правило останова:
    * Считаем ошибку и значение функции потерь
    * Считаем градиент
    * Обновляем коэффициенты

In [ ]:
def gradient_descent(X, y, iters, alpha):
    n = y.shape[0] 
    Beta = np.random.rand(2)
    for i in xrange(iters):
        y_hat = X.dot(Beta)
        
        # считаем ошибку и значение функции потерь
        error = y_hat - y 
        cost = np.sum(error ** 2) / (2 * n)

        # считаем градиент
        gradient = X.T.dot(error)

        # обновляем коэффициенты
        Beta = Beta - alpha / n * gradient  # update
    return Beta

In [ ]:
Beta = gradient_descent(X, y, 1000, 0.05)

Изобразим функцию потерь

In [ ]:
# Возьмем значения для коэффициентов на интервале от 0 до 10
beta0, beta1 = np.meshgrid(np.linspace(0, 10, 100), np.linspace(0, 10, 100))
BetaArr = np.r_[beta0.reshape(1,-1), beta1.reshape(1,-1)]

# Посчитаем ошибку от всевозможных паросочетаний
y_hat = X.dot(BetaArr)
error = y_hat - y.reshape(-1,1)
cost = np.sum(error ** 2, 0) / (2 * n)
cost = cost.reshape(beta0.shape)

plt.figure(figsize=(6,6))
contour = plt.contour(beta0, beta1, cost)
plt.clabel(contour, inline=1, fontsize=10)
plt.xlabel('$\\beta_0$')
plt.ylabel('$\\beta_1$')

## Задание

* Добавьте правило останова, срабатывающее при слабом изменении функции потерь
* Измените функцию градиентного спуска так, чтобы темп обучения мог меняться динамически 
* Дополните функцию, чтобы она так же выводила значения коэффициентов и функции потерь на некоторых итерациях. Выведите "прогресс" спуска на график выше


# Градиентный спуск с батч-оптимизацией

Теперь рассмотрим случай, когда данных в выборке много. 

В таких случаях используется стохастическая или батч-оптимизация. Первая состоит в том, что на каждом шаге итерации берется один объект, вторая — в том, что берется некоторое небольшое фиксированное количество объектов.

Загрузите данные из файла `space_ga.csv` и нормализуйте их. Мы будем предсказывать первый столбец по шести остальным. Эти данные получены с выборов в США в 1980 году. Подробнее о столбцах можно прочитать [тут](http://mldata.org/repository/data/viewslug/statlib-20050214-space_ga/1)

Как вы могли заметить, датасет больше предыдущего. На нём мы попробуем батч-оптимизацию.

## Задание
Измените функцию для градиентного спуска так, чтобы на вход они принимала дополнительный параметр — размер батча. Для простоты проверки рекомендуется изменять копию функции, реализованной выше, с измененным именем. Прокомментируйте результаты.

**Замечания**<br/>
* Объекты нужно сначала перемешать (`sklearn.utils.shuffle`), а затем разделить на батчи
* Учитите, что ошибка (и, соответственно, градиент) считается об объектам попавшим в батч, а значение функции потерь - по всем объектам.

# Другие изощренные модификации

#### Метод накопленного импульса

$$ v_t = \gamma v_{t-1} + \alpha\nabla_\beta L(\beta)$$
$$ \beta = \beta - v_t $$

#### Метод Нестерова

$$ v_t = \gamma v_{t-1} + \alpha\nabla_\beta L(\beta - \gamma v_{t-1})$$
$$ \beta = \beta - v_t $$

#### Адаптивный градиент

$g_t$ - производная по одному из параметров $\frac{\partial L}{\partial \beta_j}$

$$ G_t = G_t + g_t^2$$
$$ \beta_j = \beta_j - \frac{\alpha}{\sqrt{G_t + \epsilon}} g_t$$